In [0]:
from time import sleep
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

In [0]:
import numpy as np
import pandas as pd

In [0]:
option = Options()
option.add_argument('--incognito')
option.add_argument('--headless')
delay = 5 # 5 sec delay for a page to load

def init_browser(url):
    browser = webdriver.Chrome(executable_path = r"C:\Chrome\chromedriver.exe", chrome_options = option)
    browser.wait = WebDriverWait(browser,delay)
    browser.get(url)
    return browser

In [0]:
# Let's look for places in Japan
url = 'https://www.triposo.com/loc/Brunei/all'
browser = init_browser(url)

C:\Users\pdeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


In [0]:
# Make a dictionary of pages in Japan
places = browser.find_elements_by_class_name('feature-title')
URL_JP ={}
for place in places:
    #title = place.find_element_by_tag_name("h1")
    
    hlink = place.find_element_by_xpath(".//a")
    url = hlink.get_attribute('href')
    title = hlink.text


    URL_JP[title] = url


In [0]:
len(URL_JP)

8

In [0]:
browser.quit()
sleep(5)

In [0]:
depth_1 = {}

counter = 1
for key, value in URL_JP.items():
    
    
    counter = counter + 1
    
    cur_url = value + '/sightseeing'
    des = requests.get(cur_url)
    content = des.text
    
    if len(content)>0:
        
        if (counter%25 ==0):
            print('POI {}: Now looking for {}!'.format(counter,key))
            
        depth_2 = {}
        soup = BeautifulSoup(content)
        elems = soup.find_all(class_= "feature-title")
        sights = []
        for elm in elems:
            
            depth_3 = {}
            
            depth_3['title'] = elm.a.text
            depth_3['poi_type'] = elm.h4.text
            depth_3['sub_url'] = elm.a['href']
            
            poi_url = 'https://www.triposo.com' + elm.a['href']
            poi_des = requests.get(poi_url)
            my_info = []
            
            if poi_des.status_code == 200:
                poi_text = poi_des.text
                poi_soup = BeautifulSoup(poi_text,'lxml')
                el_poi = poi_soup.find_all(class_= "article-content")
                
                for epo in el_poi:
                    my_info.append(epo.text)
            
            depth_3['info'] = my_info
            #print(depth_3)
        
            depth_2[elm.a.text] = depth_3
        
    else:
        pass
    
    depth_1[key] = depth_2
    

In [0]:
depth_1.keys()

dict_keys(['Angkor Archaeological Park', 'Ang Trapaing Thmor', 'Anlong Veng', 'Banlung', 'Banteay Chhmar', 'Battambang', 'Bokor National Park', 'Botum Sakor National Park', 'Central Cardamom Mountains', 'Dong Peng', 'Kampong Cham', 'Kampong Chhnang', 'Kampong Speu', 'Kampong Thom', 'Kampot', 'Kep', 'Kep National Park', 'Kirirom National Park', 'Koh Ker', 'Koh Kong', 'Koh Rong', 'Koh Rung Sanloem', 'Koh Ta Kiev', 'Koh Tang', 'Koh Thonsáy', 'Kratie', 'Kulen Promtep Wildlife Sanctuary', 'Lomphat Wildlife Sanctuary', 'Oudong', 'Pailin', 'Peam Krasop Wildlife Sanctuary', 'Phnom Kulen National Park', 'Phnom Penh', 'Poipet', 'Preah Vihear', 'Pursat', 'Ream National Park', 'Samraong', 'Senmonorom', 'Siem Pang Protected Forest', 'Siem Reap', 'Sihanoukville', 'Sisophon', 'Stung Treng', 'Tonlé Sap Biosphere Reserve', 'Virachey National Park'])

In [0]:
depth_1['Ura']['Ura Monastery']

{'title': 'Ura Monastery',
 'poi_type': 'monastery',
 'sub_url': '/poi/T__5ba86d1d1d6d',
 'info': ['\nUra Monastery is a Buddhist monastery in Bhutan.\n',
  '\nWikipedia\n']}

In [0]:
# Let's find out a list of all types of attractions
all_type = {}
for key, values in depth_1.items():
    for poi, des in values.items():
        try:
            ptype = des['poi_type']
        except:
            ptype = 'nan'
        if ptype in all_type.keys():
            all_type[ptype] = all_type[ptype] + 1
        else:
            all_type[ptype] = 1
print('Number of listed type of POI in Phil is: {}'.format(len(all_type)))

Number of listed type of POI in Phil is: 27


In [0]:
all_type

{'temple': 63,
 'neighbourhood': 3,
 'bridge': 5,
 'ruin': 23,
 'sight': 80,
 'memorial': 13,
 'waterfall': 9,
 'art gallery': 2,
 'park': 2,
 'statue': 4,
 'museum': 15,
 'market': 3,
 'mountain': 8,
 'aquarium': 1,
 'island': 2,
 'dam': 1,
 'palace': 2,
 'mosque': 1,
 'church': 1,
 'lake': 1,
 'area': 3,
 'city hall': 1,
 'zoo': 1,
 'pagoda': 1,
 'cemetery': 1,
 'view point': 1,
 'protected area': 1}

In [0]:
import string

from collections import defaultdict
import json
import unicodedata
import mistune
from bs4 import BeautifulSoup # xml data parsing
import re # Regular expressions

# import xmltodict # @deepfly : you may need to install this package in anaconda environment
# xmltodict is a library that aims at making XML feel like working with JSON.

In [0]:
# Some data cleaning
def rmdiacritics(char):
    '''
    Return the base character of char, by "removing" any
    diacritics like accents or curls and strokes and the like.
    '''
    try:
        desc = unicodedata.name(char)
        cutoff = desc.find(' WITH ')
        if cutoff != -1:
            desc = desc[:cutoff]
        result = unicodedata.lookup(desc)
    except ValueError:
        result = char
    except KeyError:
        result = char
    return result


def remove_accents(input_str):
    nkfd_form = unicodedata.normalize('NFKD', input_str)
    #print(nkfd_form)
    value = u"".join([rmdiacritics(c) for c in nkfd_form])
    value = unicodedata.normalize('NFKD', value).encode('ascii','ignore')
    return value.decode('utf-8')

# Create dictionary of titles that have concatenated names as the values. For example: San Francisco: San_Francisco
def clean_word(title):
    
    value = remove_accents(title)

    #value = value.replace(' ','_')
    #value = value.encode()
    return value

In [0]:
#################
def clean_paragraph(string):
    # get rid of https:
    m = re.sub(r'(https?://\S+\ +\S+)', '', string)
    # get rid of [File:stuff]
    m = re.sub(r'\[File:[^)]*\]','', m)
    # get rid of \n
    m = re.sub(r'(\n)','', m)
    
    # get rid of ==stuff==
    m = re.sub(r'(\=+[^)]+\=+)','', m)
    #remove accents
    m = remove_accents(m)
    #get rid of ], [, '. E.g. [[Paris]] --> Paris
    m = re.sub(r'[\[\]\']','',m).strip()

    return m
###############
def clean_sentence(string):
    # punctuation and numbers to be removed
    punctuation1 = re.compile(r'[*.?!,"0-9]')
    punctuation2 = re.compile(r'[-:;()|]')
    sent = re.sub(punctuation1,'', string)
    sent = re.sub(punctuation2, ' ', sent)
    return sent

In [0]:
import nltk
nltk.download('punkt') # @ deepfly: may need to download this. Once downloaded, you can comment out this section!
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pdeep\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [0]:
# Now, let us create another dictonary where keys are Point Of Interest (POI) 
# Remove accents
mydb = {}

for key, values in depth_1.items():
    cKey = clean_word(key).lower()
    depth_2 = {}
    for poi, des in values.items():
        depth_3 = {}
        ctype = des['poi_type']
        cname = clean_word(des['title'])
        text = des['info']
        ctext = []
        for line in text: 
            line = remove_accents(line).lower()
            #convert sentence to list of words
            word_list = line.split()
            word_list = ' '.join(word_list)
            ctext.append(word_list)
        
        final_text = ' '.join(ctext)
        
        
        
        
        depth_3['poi'] = cname
        depth_3['type'] = ctype
        depth_3['text'] = final_text
        
        depth_2[cname] = depth_3
        
    mydb[cKey] = depth_2

In [0]:
len(sen)

29401

In [0]:
with open('poi_Cambodia.json','w') as f:
    json.dump(depth_1,f)

In [0]:
texts = []
for line in sen: 
    line = remove_accents(line).lower()

    #convert sentence to list of words
    word_list = line.split()
    word_list = ' '.join(word_list)

    texts.append(word_list)
final_text = ' '.join(texts)
print(final_text)

the edo-tokyo museum is a museum of the history of tokyo during the edo period. it was established in 1993. the main features of the permanent exhibitions are the life-size replica of the nihonbashi, which was the bridge leading into edo; the nakamuraza theatre; and scale models of towns and buildings from the edo, meiji and showa periods.the museum is adjacent to the ryogoku kokugikan. it was designed by kiyonori kikutake. the distinctive elevated shape of the museum building is modelled after an old storehouse in the kurazukuri style.the edo-tokyo open air architectural museum is a branch of the edo-tokyo museum. wikipedia, openstreetmap, wikivoyage, facebook api


In [0]:
final_text

'the edo-tokyo museum is a museum of the history of tokyo during the edo period. it was established in 1993. the main features of the permanent exhibitions are the life-size replica of the nihonbashi, which was the bridge leading into edo; the nakamuraza theatre; and scale models of towns and buildings from the edo, meiji and showa periods.the museum is adjacent to the ryogoku kokugikan. it was designed by kiyonori kikutake. the distinctive elevated shape of the museum building is modelled after an old storehouse in the kurazukuri style.the edo-tokyo open air architectural museum is a branch of the edo-tokyo museum. wikipedia, openstreetmap, wikivoyage, facebook api'

In [0]:
## Testing

try_page = requests.get('https://www.triposo.com/poi/N__243827989')

In [0]:
try_soup = BeautifulSoup(try_page.text)

In [0]:
elems = try_soup.find_all(class_=details)

In [0]:
print(elems.text)

History
